In [1]:
#!/usr/bin/env python3
"""
RideHail Pro Matching Server with Manual Ride Status Updates
Full-fledged script: Firebase Firestore integration, driver-passenger matching,
proposal generation, ride finalization, manual status updates, cleanup scheduler.
"""

import firebase_admin
from firebase_admin import credentials, firestore, initialize_app, get_app
from firebase_admin.firestore import GeoPoint
from math import radians, sin, cos, sqrt, atan2
import time
import os
import sys
import random
from datetime import datetime, timedelta
import threading

# ---------------- CONFIGURATION ----------------

PASSENGER_DB_CREDENTIALS = 'passenger-ride-app-firebase-adminsdk.json'
DRIVER_DB_CREDENTIALS = 'rider-ba88e-firebase-adminsdk.json'

MAX_MATCH_DISTANCE_KM = 10.0
MAX_DESTINATION_DEVIATION_KM = 5.0
DYNAMIC_PICKUP_THRESHOLD_KM = 2.0

# Database Collections
RIDER_COLLECTION_NAME = 'riders'
PROPOSAL_COLLECTION_NAME = 'driver_proposals'
PUBLIC_RIDE_REQUESTS_COLLECTION = 'public_ride_requests'

# Ride Request States (Passenger DB)
STATUS_PENDING = 'pending'
STATUS_PROPOSED = 'proposed_to_driver'
STATUS_ACCEPTED = 'accepted'
STATUS_ARRIVED_AT_PICKUP = 'arrived_at_pickup'
STATUS_PICKED_UP = 'picked_up'
STATUS_ON_WAY = 'on_way'
STATUS_COMPLETED = 'completed'
STATUS_CANCELLED = 'cancelled'

# Rider Status States (Driver DB)
RIDER_AVAILABLE = 'available'
RIDER_ON_TRIP = 'on_trip'
RIDER_OFFLINE = 'offline'

# Proposal Status States
PROPOSAL_PENDING = 'pending_acceptance'
PROPOSAL_ACCEPTED = 'accepted'
PROPOSAL_REJECTED = 'rejected'

# ----------------- FIREBASE INITIALIZATION -----------------

def initialize_firebase_app(cred_path, name):
    if not os.path.exists(cred_path):
        print(f"Error: Firebase credential file not found at {cred_path}", file=sys.stderr)
        return None
    try:
        cred = credentials.Certificate(cred_path)
        app = None
        try:
            app = firebase_admin.get_app(name)
        except ValueError:
            app = initialize_app(cred, name=name)
        db_client = firestore.client(app)
        print(f"[INIT] Firebase DB '{name}' initialized successfully.")
        return db_client
    except Exception as e:
        print(f"[ERROR] Initializing Firebase App '{name}': {e}", file=sys.stderr)
        return None

# ----------------- UTILITY FUNCTIONS -----------------

def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371
    lat1_rad = radians(lat1)
    lon1_rad = radians(lon1)
    lat2_rad = radians(lat2)
    lon2_rad = radians(lon2)
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    a = sin(dlat / 2)**2 + cos(lat1_rad)*cos(lat2_rad)*sin(dlon/2)**2
    c = 2*atan2(sqrt(a), sqrt(1-a))
    return R * c

def calculate_dynamic_pickup(driver_loc, passenger_loc):
    dist = calculate_distance(driver_loc.latitude, driver_loc.longitude,
                              passenger_loc.latitude, passenger_loc.longitude)
    if dist < DYNAMIC_PICKUP_THRESHOLD_KM:
        mid_lat = (driver_loc.latitude + passenger_loc.latitude)/2
        mid_lon = (driver_loc.longitude + passenger_loc.longitude)/2
        return GeoPoint(mid_lat, mid_lon)
    return passenger_loc

def check_destination_overlap(driver_dest, passenger_dest):
    dist = calculate_distance(driver_dest.latitude, driver_dest.longitude,
                              passenger_dest.latitude, passenger_dest.longitude)
    return dist <= MAX_DESTINATION_DEVIATION_KM

def calculate_match_score(distance_m, rating, passenger_preference, rider_vehicle_type, total_rides):
    distance_score = 1.0 - (distance_m / 10000)
    rating_score = rating/5.0
    experience_score = min(total_rides/100.0, 1.0)
    preference_score = 1.0 if (passenger_preference=='Any' or passenger_preference.lower() in rider_vehicle_type.lower()) else 0.5
    return distance_score*0.4 + rating_score*0.3 + experience_score*0.2 + preference_score*0.1

def calculate_priority_level(match_score):
    if match_score >= 0.8:
        return 1
    elif match_score >= 0.6:
        return 2
    else:
        return 3

def calculate_fare(distance_km, ride_type):
    base_fare = 30.0
    rate_per_km = 8.0
    if ride_type == 'Premium':
        base_fare = 50.0
        rate_per_km = 12.0
    elif ride_type == 'SUV':
        base_fare = 70.0
        rate_per_km = 15.0
    elif ride_type == 'Electric':
        base_fare = 35.0
        rate_per_km = 7.0
    return base_fare + (distance_km*rate_per_km)

def generate_otp():
    return str(random.randint(1000, 9999))

# ----------------- MATCHING & PROPOSAL -----------------

def _create_rider_proposal(pdb, ddb, ride_id, request_data, rider_info):
    try:
        passenger_pickup_loc = request_data.get('pickupLocation')
        passenger_dest_loc = request_data.get('destinationLocation')
        ride_distance = calculate_distance(passenger_pickup_loc.latitude, passenger_pickup_loc.longitude,
                                           passenger_dest_loc.latitude, passenger_dest_loc.longitude)
        fare_amount = calculate_fare(ride_distance, request_data.get('rideType','Standard'))
        otp = generate_otp()
        proposal_data = {
            'original_request_id': ride_id,
            'riderUid': rider_info['rider_id'],
            'riderName': rider_info['rider_name'],
            'riderPhone': rider_info['rider_phone'],
            'riderRating': rider_info['rating'],
            'vehicleNumber': rider_info['vehicle_number'],
            'vehicleModel': rider_info['vehicle_model'],
            'vehicleColor': rider_info['vehicle_color'],
            'vehicleType': rider_info['vehicle_type'],
            'totalRides': rider_info['total_rides'],
            'passengerId': request_data.get('passengerId'),
            'passengerName': request_data.get('passengerName','Passenger'),
            'passengerPhone': request_data.get('passengerPhone','N/A'),
            'pickupLocation': request_data.get('pickupLocation'),
            'destinationLocation': request_data.get('destinationLocation'),
            'pickupAddress': request_data.get('pickupAddress','Pickup Location'),
            'destinationAddress': request_data.get('destinationAddress','Destination Location'),
            'fareAmount': fare_amount,
            'paymentMethod': request_data.get('paymentMethod','Cash'),
            'rideType': request_data.get('rideType','Standard'),
            'passengerRating': request_data.get('passengerRating',5.0),
            'estimatedDistance': f"{ride_distance:.1f} km",
            'estimatedDuration': f"{(ride_distance/40*60):.0f} min",
            'specialRequests': request_data.get('specialRequests','None'),
            'vehiclePreference': request_data.get('vehiclePreference','Any'),
            'luggageCount': request_data.get('luggageCount',0),
            'passengerCount': request_data.get('passengerCount',1),
            'status': PROPOSAL_PENDING,
            'requestTimestamp': firestore.SERVER_TIMESTAMP,
            'distanceToPickup': rider_info['distance_to_pickup']*1000,
            'match_score': rider_info['match_score'],
            'priority_level': rider_info['priority_level'],
            'otp': otp,
            'otpVerified': False,
            'createdAt': firestore.SERVER_TIMESTAMP,
            'proposalExpiresAt': firestore.SERVER_TIMESTAMP
        }
        proposal_ref = ddb.collection(PROPOSAL_COLLECTION_NAME).document()
        proposal_ref.set(proposal_data)
        print(f"[PROPOSAL] Sent to {rider_info['rider_name']} | Score: {rider_info['match_score']:.2f} | OTP: {otp}")
    except Exception as e:
        print(f"[ERROR] Failed to create proposal for {rider_info['rider_id']}: {e}")

def match_driver(pdb, ddb, ride_request_snapshot):
    ride_id = ride_request_snapshot.id
    request_data = ride_request_snapshot.to_dict()
    passenger_pickup_loc = request_data.get('pickupLocation')
    passenger_dest_loc = request_data.get('destinationLocation')
    if not (passenger_pickup_loc and passenger_dest_loc):
        print(f"[ERROR] Ride {ride_id} missing location data. Skipping.")
        return
    available_riders_query = ddb.collection(RIDER_COLLECTION_NAME).where('status','==',RIDER_AVAILABLE)
    potential_riders = available_riders_query.stream()
    rider_list = list(potential_riders)
    suitable_riders = []
    for rider_doc in rider_list:
        rider_data = rider_doc.to_dict()
        rider_id = rider_doc.id
        rider_loc = rider_data.get('currentLocation')
        rider_dest = rider_data.get('currentRouteEnd')
        if not rider_loc:
            continue
        distance_to_pickup = calculate_distance(rider_loc.latitude,rider_loc.longitude,
                                                passenger_pickup_loc.latitude,passenger_pickup_loc.longitude)
        if distance_to_pickup > MAX_MATCH_DISTANCE_KM:
            continue
        if rider_dest:
            if not check_destination_overlap(rider_dest, passenger_dest_loc):
                continue
        rider_vehicle_type = rider_data.get('vehicleType','')
        vehicle_preference = request_data.get('vehiclePreference','Any')
        if vehicle_preference != 'Any' and vehicle_preference.lower() not in rider_vehicle_type.lower():
            continue
        match_score = calculate_match_score(distance_to_pickup*1000, rider_data.get('rating',5.0),
                                            vehicle_preference, rider_vehicle_type, rider_data.get('total_rides',0))
        if match_score >= 0.6:
            suitable_riders.append({
                'rider_id': rider_id,
                'rider_name': rider_data.get('riderName',rider_data.get('name','Rider')),
                'rider_phone': rider_data.get('phone','N/A'),
                'rating': rider_data.get('rating',5.0),
                'total_rides': rider_data.get('total_rides',0),
                'vehicle_type': rider_vehicle_type,
                'vehicle_number': rider_data.get('vehicle_number'),
                'vehicle_model': rider_data.get('vehicle_model'),
                'vehicle_color': rider_data.get('vehicle_color'),
                'distance_to_pickup': distance_to_pickup,
                'match_score': match_score,
                'priority_level': calculate_priority_level(match_score)
            })
    suitable_riders.sort(key=lambda x: x['match_score'], reverse=True)
    for rider_info in suitable_riders[:5]:
        _create_rider_proposal(pdb, ddb, ride_id, request_data, rider_info)

# ----------------- FINALIZATION & STATUS UPDATE -----------------

def finalize_match(pdb, ddb, proposal_data):
    ride_id = proposal_data['original_request_id']
    rider_id = proposal_data['riderUid']
    try:
        ride_ref = pdb.collection(PUBLIC_RIDE_REQUESTS_COLLECTION).document(ride_id)
        ride_ref.update({
            'status': STATUS_ACCEPTED,
            'riderUid': rider_id,
            'riderName': proposal_data.get('riderName','Rider'),
            'riderPhone': proposal_data.get('riderPhone','N/A'),
            'riderRating': proposal_data.get('riderRating',5.0),
            'vehicleNumber': proposal_data.get('vehicleNumber'),
            'vehicleModel': proposal_data.get('vehicleModel'),
            'vehicleColor': proposal_data.get('vehicleColor'),
            'vehicleType': proposal_data.get('vehicleType'),
            'totalRides': proposal_data.get('totalRides',0),
            'matched_at': firestore.SERVER_TIMESTAMP
        })
        rider_ref = ddb.collection(RIDER_COLLECTION_NAME).document(rider_id)
        rider_ref.update({
            'status': RIDER_ON_TRIP,
            'current_passenger_id': proposal_data.get('passengerId'),
            'current_ride_id': ride_id,
            'lastActive': firestore.SERVER_TIMESTAMP
        })
        proposal_query = ddb.collection(PROPOSAL_COLLECTION_NAME).where('original_request_id','==',ride_id)
        proposals = proposal_query.get()
        for proposal in proposals:
            proposal.reference.update({
                'status': PROPOSAL_ACCEPTED,
                'acceptedTimestamp': firestore.SERVER_TIMESTAMP
            })
        print(f"[MATCHED] Ride {ride_id} matched with Rider {rider_id}")
    except Exception as e:
        print(f"[ERROR] Finalization failed for ride {ride_id}: {e}")

def handle_ride_status_update(pdb, ddb, ride_id, new_status):
    try:
        ride_ref = pdb.collection(PUBLIC_RIDE_REQUESTS_COLLECTION).document(ride_id)
        update_data = {'status': new_status, 'lastUpdated': firestore.SERVER_TIMESTAMP}
        if new_status == STATUS_ARRIVED_AT_PICKUP:
            update_data['arrivalTimestamp'] = firestore.SERVER_TIMESTAMP
        elif new_status == STATUS_PICKED_UP:
            update_data['pickupTimestamp'] = firestore.SERVER_TIMESTAMP
            update_data['otpVerified'] = True
        elif new_status == STATUS_COMPLETED:
            update_data['completionTimestamp'] = firestore.SERVER_TIMESTAMP
        elif new_status == STATUS_CANCELLED:
            update_data['cancellationTimestamp'] = firestore.SERVER_TIMESTAMP
        ride_ref.update(update_data)
        print(f"[STATUS] Ride {ride_id} updated to '{new_status}'")
    except Exception as e:
        print(f"[ERROR] Updating ride status failed: {e}")

# ----------------- SNAPSHOT LISTENERS -----------------

def on_new_ride_request_snapshot(col_snapshot, changes, read_time):
    for change in changes:
        if change.type.name in ('ADDED','MODIFIED'):
            data = change.document.to_dict()
            if data and data.get('status') == STATUS_PENDING:
                print(f"[NEW RIDE] Detected Ride ID: {change.document.id}")
                match_driver(db_passenger, db_driver, change.document)

def on_proposal_update_snapshot(col_snapshot, changes, read_time):
    for change in changes:
        if change.type.name == 'MODIFIED':
            data = change.document.to_dict()
            current_status = data.get('status')
            if current_status == PROPOSAL_ACCEPTED:
                data['proposal_id'] = change.document.id
                finalize_match(db_passenger, db_driver, data)

def on_rider_status_snapshot(col_snapshot, changes, read_time):
    for change in changes:
        if change.type.name in ('ADDED','MODIFIED'):
            data = change.document.to_dict()
            rider_id = change.document.id
            status = data.get('status','offline')
            loc = data.get('currentLocation')
            if status == RIDER_AVAILABLE and loc:
                print(f"[RIDER] {rider_id} available at ({loc.latitude:.4f},{loc.longitude:.4f})")
            elif status == RIDER_ON_TRIP:
                print(f"[RIDER] {rider_id} is on trip")
            elif status == RIDER_OFFLINE:
                print(f"[RIDER] {rider_id} went offline")

# ----------------- CLEANUP -----------------

def cleanup_old_proposals():
    try:
        one_hour_ago = datetime.now() - timedelta(hours=1)
        proposals_query = db_driver.collection(PROPOSAL_COLLECTION_NAME)
        old_proposals = proposals_query.get()
        for proposal in old_proposals:
            pdata = proposal.to_dict()
            ts = pdata.get('requestTimestamp')
            if ts and hasattr(ts,'timestamp'):
                if datetime.fromtimestamp(ts.timestamp()) < one_hour_ago:
                    if pdata.get('status') in [PROPOSAL_PENDING, PROPOSAL_REJECTED]:
                        proposal.reference.delete()
                        print(f"[CLEANUP] Deleted old proposal {proposal.id}")
    except Exception as e:
        print(f"[CLEANUP ERROR] {e}")

def schedule_cleanup():
    while True:
        time.sleep(3600)
        cleanup_old_proposals()

# ----------------- MANUAL STATUS INPUT -----------------

def manual_status_input():
    while True:
        try:
            print("\n=== Manual Ride Status Update ===")
            ride_id = input("Enter Ride ID (or 'exit'): ").strip()
            if ride_id.lower() == 'exit':
                break
            print("Available statuses: arrived_at_pickup, picked_up, on_way, completed, cancelled")
            status = input("Enter new status: ").strip()
            if status not in [STATUS_ARRIVED_AT_PICKUP, STATUS_PICKED_UP, STATUS_ON_WAY, STATUS_COMPLETED, STATUS_CANCELLED]:
                print("[ERROR] Invalid status. Try again.")
                continue
            handle_ride_status_update(db_passenger, db_driver, ride_id, status)
        except Exception as e:
            print(f"[ERROR] {e}")

# ----------------- MAIN EXECUTION -----------------

if __name__ == '__main__':
    db_passenger = initialize_firebase_app(PASSENGER_DB_CREDENTIALS, 'passenger_app')
    db_driver = initialize_firebase_app(DRIVER_DB_CREDENTIALS, 'driver_app')
    if not (db_passenger and db_driver):
        print("[FATAL] DB connection failed. Exiting.")
        sys.exit(1)
    print("[SERVER] RideHail Pro Matching Server Started.")
    # Listeners
    db_passenger.collection(PUBLIC_RIDE_REQUESTS_COLLECTION).where('status','==',STATUS_PENDING).on_snapshot(on_new_ride_request_snapshot)
    db_driver.collection(PROPOSAL_COLLECTION_NAME).on_snapshot(on_proposal_update_snapshot)
    db_driver.collection(RIDER_COLLECTION_NAME).on_snapshot(on_rider_status_snapshot)
    # Start cleanup thread
    threading.Thread(target=schedule_cleanup, daemon=True).start()
    # Manual status input in main thread
    manual_status_input()


[FATAL] DB connection failed. Exiting.


Error: Firebase credential file not found at passenger-ride-app-firebase-adminsdk.json
Error: Firebase credential file not found at rider-ba88e-firebase-adminsdk.json


SystemExit: 1

C:\Users\DELL\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3554: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
